In [33]:
import os
import pandas as pd
import sys
import nltk
EN_TOKENIZER = nltk.data.load("tokenizers/punkt/english.pickle")
cwd = os.getcwd()
sys.path.append(cwd+'/../POLIANNA') # directs to POLIANNA GitHub repo (for unpickling info contained in src dir)
pol_dir = cwd+"/../../PhD/POLIANNA_v1_1/POLIANNA_v1_1" # directs to POLIANNA dataset main dir
pkl_add = pol_dir+"/02_processed_to_dataframe/preprocessed_dataframe.pkl"
pann_df = pd.read_pickle(pkl_add)

We only care about the text, tokens and curation.

In [34]:
pann_df2 = pann_df[['Text', 'Tokens', "Curation"]].copy()
pann_df2.index=range(len(pann_df2))

Currently each text consists of an article made up of multiple sentences. At some point we may want to update the dataset to be sentence-level.

In [35]:
idx = 0
txt = pann_df2.loc[idx,"Text"]
tok = pann_df2.loc[idx,"Tokens"]
cur = pann_df2.loc[idx,"Curation"]
print(len(txt), len(tok), len(cur))
#snts = EN_TOKENIZER.tokenize(txt)

2209 458 36


In [36]:
print(tok[0])
print(cur[0])
x = str(cur[0]).split("feature:")[1]
feature = x.split(" tag:")[0]
x = str(cur[0]).split("tag:")[1]
tag = x.split(" start:")[0]
feature, tag

token id: T1, start:0 stop:7 text:article tag_count:0
span id:CUR0 annotator:Curation layer:Instrumenttypes feature:InstrumentType tag:Unspecified start:64 stop:78 text:delegated acts


('InstrumentType', 'Unspecified')

In [37]:
#IOB2
newdct = {}
for article in pann_df2.index:
    tok_entry = pann_df2.loc[article,"Tokens"]
    tokens = []
    for tok in tok_entry:
        tokens.append(
            {"start":str(tok).split("start:")[1].split(" stop:")[0],
             "stop":str(tok).split("stop:")[1].split(" text:")[0],
             "text":str(tok).split("text:")[1].split(" tag_count:")[0]}
        )
    tok_lst = [token["text"] for token in tokens]
    spn_entry = pann_df2.loc[article,"Curation"]
    spans = []
    for spn in spn_entry:
        ftr = str(spn).split("feature:")[1].split("tag:")[0][:-1]
        if ftr == "InstrumentType":
            spans.append(
                {
                    "start":str(spn).split("start:")[1].split(" stop:")[0],
                    "stop":str(spn).split("stop:")[1].split(" text:")[0],
                    "feature":ftr,
                    "tag":str(spn).split("tag:")[1].split(" start:")[0]
                }
            )
    bn_tags = ["O"]*len(tokens)
    mc_tags = ["O"]*len(tokens)
    for span in spans:
        bn_label = span["feature"]
        mc_label = span["tag"]
        for i, token in enumerate(tokens):
            if token["stop"] <= span["start"]:
                continue
            if token["start"] >= span["stop"]:
                break
            if span["start"] <= token["start"] < span["stop"]:
                bn_tags[i] = "B-" + bn_label if token["start"] == span["start"] else "I-" + bn_label
                mc_tags[i] = "B-" + mc_label if token["start"] == span["start"] else "I-" + mc_label
    newdct[article] = {
        "text_tokens":tok_lst,
        "bio_tags_bn":bn_tags,
        "bio_tags_mc":mc_tags
    }

In [38]:
for i in range(len(newdct[0]["text_tokens"])):
    print(newdct[0]["text_tokens"][i], newdct[0]["bio_tags_bn"][i], newdct[0]["bio_tags_mc"][i])

article O O
43 O O
exercise O O
of O O
the O O
delegation O O
1 O O
. O O
    O O
the O O
power O O
to O O
adopt O O
delegated B-InstrumentType B-Unspecified
acts I-InstrumentType I-Unspecified
is O O
conferred O O
on O O
the O O
commission O O
subject O O
to O O
the O O
conditions O O
laid O O
down O O
in O O
this O O
article O O
. O O
2 O O
. O O
    O O
the O O
power O O
to O O
adopt O O
delegated O O
acts O O
referred O O
to O O
in O O
articles O O
3 O O
( O O
5 O O
) O O
, O O
article O O
15 O O
( O O
5 O O
) O O
, O O
article O O
26 O O
( O O
6 O O
) O O
, O O
article O O
37 O O
( O O
7 O O
) O O
and O O
article O O
40 O O
( O O
4 O O
) O O
shall O O
be O O
conferred O O
on O O
the O O
commission O O
for O O
a O O
period O O
of O O
five O O
years O O
from O O
24 O O
  O O
december O O
2018 O O
. O O
the O O
commission O O
shall O O
draw O O
up O O
a O O
report O O
in O O
respect O O
of O O
the O O
delegation O O
of O O
power O O
not O O
later O O
than O O
nine O O
months O O
befo

In [39]:
#feature
id2label_bn = {
    0: "O",
    1: "B-InstrumentType",
    2: "I-InstrumentType"
}
label2id_bn = {
    "O":0,
    "B-InstrumentType":1,
    "I-InstrumentType":2
}

#tag
id2label_mc = {
    0:"O",
    1:"B-Edu_Outreach",
    2:"I-Edu_Outreach",
    3:"B-FrameworkPolicy",
    4:"I-FrameworkPolicy",
    5:"B-PublicInvt",
    6:"I-PublicInvt",
    7:"B-RD_D",
    8:"I-RD_D",
    9:"B-RegulatoryInstr",
    10:"I-RegulatoryInstr",
    11:"B-Subsidies_Incentives",
    12:"I-Subsidies_Incentives",
    13:"B-TaxIncentives",
    14:"I-TaxIncentives",
    15:"B-TradablePermit",
    16:"I-TradablePermit",
    17:"B-Unspecified",
    18:"I-Unspecified",
    19:"B-VoluntaryAgrmt",
    20:"I-VoluntaryAgrmt"
}
label2id_mc = {
    "O":0,
    "B-Edu_Outreach":1,
    "I-Edu_Outreach":2,
    "B-FrameworkPolicy":3,
    "I-FrameworkPolicy":4,
    "B-PublicInvt":5,
    "I-PublicInvt":6,
    "B-RD_D":7,
    "I-RD_D":8,
    "B-RegulatoryInstr":9,
    "I-RegulatoryInstr":10,
    "B-Subsidies_Incentives":11,
    "I-Subsidies_Incentives":12,
    "B-TaxIncentives":13,
    "I-TaxIncentives":14,
    "B-TradablePermit":15,
    "I-TradablePermit":16,
    "B-Unspecified":17,
    "I-Unspecified":18,
    "B-VoluntaryAgrmt":19,
    "I-VoluntaryAgrmt":20
}

In [40]:
toklst_list = []
ner_tags_list_bn = []
ner_tags_list_mc = []

for article in list(newdct):
    toklst_list.append(newdct[article]["text_tokens"])
    ner_tags_list_bn.append([label2id_bn[lbl] for lbl in newdct[article]["bio_tags_bn"]])
    ner_tags_list_mc.append([label2id_mc[lbl] for lbl in newdct[article]["bio_tags_mc"]])

print(toklst_list[0])
print(ner_tags_list_bn[0])
print(ner_tags_list_mc[0])

len(toklst_list), len(ner_tags_list_bn), len(ner_tags_list_mc)

['article', '43', 'exercise', 'of', 'the', 'delegation', '1', '.', '\xa0\xa0\xa0', 'the', 'power', 'to', 'adopt', 'delegated', 'acts', 'is', 'conferred', 'on', 'the', 'commission', 'subject', 'to', 'the', 'conditions', 'laid', 'down', 'in', 'this', 'article', '.', '2', '.', '\xa0\xa0\xa0', 'the', 'power', 'to', 'adopt', 'delegated', 'acts', 'referred', 'to', 'in', 'articles', '3', '(', '5', ')', ',', 'article', '15', '(', '5', ')', ',', 'article', '26', '(', '6', ')', ',', 'article', '37', '(', '7', ')', 'and', 'article', '40', '(', '4', ')', 'shall', 'be', 'conferred', 'on', 'the', 'commission', 'for', 'a', 'period', 'of', 'five', 'years', 'from', '24', '\xa0', 'december', '2018', '.', 'the', 'commission', 'shall', 'draw', 'up', 'a', 'report', 'in', 'respect', 'of', 'the', 'delegation', 'of', 'power', 'not', 'later', 'than', 'nine', 'months', 'before', 'the', 'end', 'of', 'the', 'five-year', 'period', '.', 'the', 'delegation', 'of', 'power', 'shall', 'be', 'tacitly', 'extended', 'for'

(448, 448, 448)

In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
#
'''
for e in [0,3,6,9]:
    print(f"\nRound {e}\n")
    bn_ft_toks, bn_ho_toks, bn_ft_tags, bn_ho_tags = train_test_split(toklst_list, ner_tags_list_bn, test_size=0.2, random_state=e)
    mc_ft_toks, mc_ho_toks, mc_ft_tags, mc_ho_tags = train_test_split(toklst_list, ner_tags_list_mc, test_size=0.2, random_state=e)
    #
    bn_train_toks, bn_test_toks, bn_train_tags, bn_test_tags = train_test_split(bn_ft_toks, bn_ft_tags, test_size=0.25, random_state=e)
    mc_train_toks, mc_test_toks, mc_train_tags, mc_test_tags = train_test_split(mc_ft_toks, mc_ft_tags, test_size=0.25, random_state=e)
    #
    bn_ds = DatasetDict({
        "train": Dataset.from_list([{"ner_tags":tags,"tokens":toks} for toks, tags in zip(bn_train_toks, bn_train_tags)]),
        "test": Dataset.from_list([{"ner_tags":tags,"tokens":toks} for toks, tags in zip(bn_test_toks, bn_test_tags)]),
        "holdout": Dataset.from_list([{"ner_tags":tags,"tokens":toks} for toks, tags in zip(bn_ho_toks, bn_ho_tags)])
    })
    mc_ds = DatasetDict({
        "train": Dataset.from_list([{"ner_tags":tags,"tokens":toks} for toks, tags in zip(mc_train_toks, mc_train_tags)]),
        "test": Dataset.from_list([{"ner_tags":tags,"tokens":toks} for toks, tags in zip(mc_test_toks, mc_test_tags)]),
        "holdout": Dataset.from_list([{"ner_tags":tags,"tokens":toks} for toks, tags in zip(mc_ho_toks, mc_ho_tags)])
    })
    ############
    bn_path = cwd+f"/inputs/pol_{e}_bn"
    mc_path = cwd+f"/inputs/pol_{e}_mc"
    if not os.path.exists(bn_path):
        os.makedirs(bn_path)
    if not os.path.exists(mc_path):
        os.makedirs(mc_path)
    bn_ds.save_to_disk(bn_path)
    print(f"Saved pol_{e}_bn")
    mc_ds.save_to_disk(mc_path)
    print(f"Saved pol_{e}_mc")
'''

Training loop

In [41]:
id2label = {"bn": id2label_bn,"mc":id2label_mc}
label2id = {"bn": label2id_bn,"mc":label2id_mc}

In [42]:
import evaluate
import torch
import torch.nn as nn
import json
import numpy as np
import time
from transformers import AutoModelForTokenClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification

input_dir = cwd+"/inputs"

def finetune_polianna(datasetdct, int2label, label2int, mode, model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=f"{os.getcwd()}/outputs/models", hyperparams=False, report_to="none"):
    if not hyperparams:
        hyperparams = {
            "epochs":10, 
            "r":9,
            "lr":2e-5,
            "batch_size":16}
    epochs = hyperparams["epochs"]
    rstate = hyperparams["r"]
    lr = hyperparams["lr"]
    batch_size = hyperparams["batch_size"]
    start = time.time()
    num_lbs = len(list(int2label))
    print(f'\nLoading model {model_name}\n')
    print("Tokenizing")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    def tokenize_and_align_labels(examples):
        #Fxn from https://huggingface.co/docs/transformers/en/tasks/token_classification
        tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
        labels = []
        for i, label in enumerate(examples[f"ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:  # Set the special tokens to -100.
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs
    tokenized_test = datasetdct["test"].map(tokenize_and_align_labels, batched=True)
    tokenized_train = datasetdct["train"].map(tokenize_and_align_labels, batched=True)
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
    metric_log = []
    seqeval = evaluate.load("seqeval")
    def compute_metrics(p):
        #Fxn from https://huggingface.co/docs/transformers/en/tasks/token_classification
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)
        true_predictions = [
            [list(label2int)[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [list(label2int)[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        results = seqeval.compute(predictions=true_predictions, references=true_labels, scheme="IOB2")
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }
    model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_lbs, id2label=int2label, label2id=label2int)
    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        seed=9,
        num_train_epochs=epochs,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="best",
        logging_strategy="epoch",
        optim="adamw_torch",
        load_best_model_at_end=True,
        report_to= report_to,
        run_name=f"{output_dir.split('_')[-2]}{rstate}" if report_to == "wandb" else "X"
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    print("Training")
    trainer.train()
    train_losses = [log["loss"] for log in trainer.state.log_history if "loss" in log]
    eval_losses = [log["eval_loss"] for log in trainer.state.log_history if "eval_loss" in log]
    print("Saving")
    model_fn = f"{model_name.split('/')[-1]}_{mode}_e{epochs}_r{rstate}.pt"
    trainer.save_model(output_dir+f"/{model_fn}")
    #
    tokenized_ho = datasetdct["holdout"].map(tokenize_and_align_labels, batched=True)
    predictions = trainer.predict(tokenized_ho)
    logits = predictions.predictions
    labels = predictions.label_ids
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1).numpy()
    #
    metric_log.append({"train_loss":train_losses})
    metric_log.append({"eval_loss": eval_losses})
    metrics = compute_metrics((logits, labels))
    metric_log.append({"metrics": metrics})
    with open(output_dir+f"/{model_fn}/metrics.json", "w", encoding="utf-8") as f:
        json.dump(metric_log, f, ensure_ascii=False, indent=4)
    end = time.time()
    print(metrics)
    print(f"\nSaved {model_name.split('/')[-1]}_{mode}_e{epochs}_r{rstate}.")
    print(f'\nDone in {round((end-start)/60,2)} min')
    return metrics

In [ ]:
!pip install wandb
!wandb login ################################

In [ ]:
import gc
import wandb

# this model doesnt work well, classifies all tokens as "O"
# would splitting into sentences first help?

os.environ["WANDB_PROJECT"]="polianna_inc"
os.environ["WANDB_LOG_MODEL"]="checkpoint"
os.environ["WANDB_WATCH"]="false"

metriclog = {}
exps = [0,3,6,9]#range(10)#[6]#[0,3,6,9]#
torch.cuda.empty_cache()
gc.collect()

for r in exps:
    output_dir = cwd+"/outputs/fting_X_bn"
    ds = DatasetDict.load_from_disk(f"{input_dir}/pol_{r}_bn")
    hyper = {
        "epochs":2, 
        "r":r, 
        "lr":2E-5,
        "batch_size":16
        }
    torch.cuda.empty_cache()
    gc.collect()
    wandbrun = wandb.init(config=hyper, group="X", name=f"X{r}", reinit='create_new')
    metrics = finetune_polianna(ds, id2label["bn"], label2id["bn"], "bn", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper, report_to="wandb")
    wandbrun.finish()
    metriclog[f'bn_{r}'] = metrics
    hyp_rpt = {"bn":hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)
for r in exps:
    output_dir = cwd+"/outputs/fting_W_mc"
    ds = DatasetDict.load_from_disk(f"{input_dir}/pol_{r}_mc")
    hyper = {
        "epochs":2, 
        "r":r, 
        "lr":2E-5,
        "batch_size":16
        }
    torch.cuda.empty_cache()
    gc.collect()
    wandbrun = wandb.init(config=hyper, group="W", name=f"W{r}", reinit='create_new')
    metrics = finetune_polianna(ds, id2label["mc"], label2id["mc"], "mc", model_name="sentence-transformers/paraphrase-xlm-r-multilingual-v1", dev='cuda', output_dir=output_dir, hyperparams=hyper, report_to="wandb")
    wandbrun.finish()
    metriclog[f'mc_{r}'] = metrics
    hyp_rpt = {"mc":hyper}
    with open(output_dir+f"/run_details.json", "w", encoding="utf-8") as f:
        json.dump(hyp_rpt, f, ensure_ascii=False, indent=4)

print(metriclog)

In [ ]:
from datasets import DatasetDict
ds = DatasetDict.load_from_disk(f"{cwd}/inputs/ds_9_mc")
pl = DatasetDict.load_from_disk(f"{cwd}/inputs/pol_9_mc")

In [52]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(cwd+"/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_bn_e2_r9.pt")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True).to("cuda") #, is_split_into_words=True
tk_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [71]:
lst_n = []
for x in range(len(pl["holdout"])):
    pl_tok = pl["holdout"][x]["tokens"]
    pl_lbl = pl["holdout"][x]["ner_tags"]
    if sum(pl_lbl)>0:
        lst_n.append(x)
print(lst_n)

[8, 10, 12, 13, 16, 20, 22, 27, 28, 31, 33, 34, 43, 45, 51, 64, 68, 69, 70, 74, 75, 76, 77, 79, 80, 83, 86, 87]


In [ ]:
# 6min
from transformers import pipeline
bn_class_pl = pipeline("ner", model=cwd+"/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_bn_e2_r9.pt")
mc_class_pl = pipeline("ner", model=cwd+"/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_mc_e2_r9.pt")
for x in lst_n:
    pl_tok = pl["holdout"][x]["tokens"]
    pl_lbl = pl["holdout"][x]["ner_tags"]
    bn_pl_out = bn_class_pl(pl_tok)
    mc_pl_out = mc_class_pl(pl_tok)
    print("\n", x)
    print("bn", bn_pl_out)
    print("mc", mc_pl_out)
    print("rl", pl_lbl)


 8
bn [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], 

In [72]:
bn_tokenizer = AutoTokenizer.from_pretrained(cwd+"/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_bn_e2_r9.pt")
mc_tokenizer = AutoTokenizer.from_pretrained(cwd+"/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_mc_e2_r9.pt")
bn_model = AutoModelForTokenClassification.from_pretrained(cwd+"/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_bn_e2_r9.pt")
mc_model = AutoModelForTokenClassification.from_pretrained(cwd+"/inputs/polianna_models/paraphrase-xlm-r-multilingual-v1_mc_e2_r9.pt")

In [79]:
x = 8
pl_tok = pl["holdout"][x]["tokens"]
pl_lbl = pl["holdout"][x]["ner_tags"]
#
bn_inputs = bn_tokenizer(" ".join(pl_tok), return_tensors="pt", truncation=True, padding=True)
mc_inputs =  mc_tokenizer(" ".join(pl_tok), return_tensors="pt", truncation=True, padding=True)
#
with torch.no_grad():
    bn_logits = bn_model(**bn_inputs).logits
bn_pred = torch.argmax(bn_logits, dim=2)
bn_pred_tok_cls = [bn_model.config.id2label[t.item()] for t in bn_pred[0]]
#
with torch.no_grad():
    mc_logits = mc_model(**mc_inputs).logits
mc_pred = torch.argmax(mc_logits, dim=2)
mc_pred_tok_cls = [mc_model.config.id2label[t.item()] for t in mc_pred[0]]
#
print(bn_pred_tok_cls)
print(mc_pred_tok_cls)
print([mc_model.config.id2label[t] for t in pl_lbl])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',